## Сравнение тарифов двух и более спонсоров из файлов
- Делается сравнение покрытия по странам
- Делается сравнения покрытия по LTE
- Делается сравнение тарифов по странам с подкраской минимального тарифа (с подкраской LTE)
- Выгружается файл сравнения
- Выполняется проверка пропавших и появившихся операторов и стран для Combined Allowed List
- Готовятся файлы для загрузки тарифов в OCS
- Готовятся файлы для загрузки тарифов в RoamDb

По результатам выполнения данной процедуры можно принимать решение об изменении Allowed List.
Для загрузки изменений Allowed List использовать методику c:\W_DATA_ROAM\МЕТОДИКИ\_NOC_M_NEW_CONNECTION\020_DMI_ALLOWED_LIST+STS\

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pyodbc
import datetime as dt
from os.path import join
import roamability as rb

In [2]:
# Из базы данных OCS выгружаем TADIG

# Connect to OCSDBREP1 (BSS)

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

# Получение данных TADIG

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME AS COUNTRY,n.NETWORK_NAME AS NETWORK,t.TADIG_CODE AS TADIG,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()

df_tadig.head(3)

,COUNTRY,NETWORK,TADIG,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0


In [3]:
# Описываем класс данных Спонсора

class SponsorClass:
    
    def __init__(self, name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid,
                 reload_tariff_from_db = False, downloads = None, tariff_file_name = None):
        self.name = name
        self.sponsor_id = sponsor_id
        self.ocs_roaming_plan_id = ocs_roaming_plan_id
        self.roamdb_tarifflistid = roamdb_tarifflistid
        self.df_ocs_cost = DataFrame()
        self.df_roamdb = DataFrame()
        
        self.downloads = downloads
        self.tariff_file_name = tariff_file_name
        
        if reload_tariff_from_db:
            self.upload_data_from_ocs()
            self.upload_data_from_roamdb()
            df_temp = pd.merge(self.df_ocs_cost, self.df_roamdb, how='left', on='TADIG')
            self.df_tariff = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        else:
            self.prepare_pp()
        
    def ocs_tariff_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из OCS'''
        sql_srt=\
        '''
        SELECT t.tadig_code AS TADIG
        ,MO_CALL_RATE MOC,MT_CALL_RATE MTC,MO_SMS_RATE SMS,DATA_RATE DATA
        FROM ROAMING_PLAN_RULES r
        LEFT JOIN RDB_TADIG_CODES t
        ON r.network_id=t.network_id
        LEFT JOIN ROAMING_PLANS p
        ON r.roaming_plan_id=p.roaming_plan_id
        WHERE r.roaming_plan_id IN ({0})
        AND r.SPONSOR_ID = {1}
        AND p.deleted IS NULL
        AND r.end_date IS NULL
        AND r.is_discounted = 1
        '''.format(self.ocs_roaming_plan_id, self.sponsor_id)
        return sql_srt
    
    def roamdb_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из RoamDB'''
        sql_srt=\
        '''
        SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
        ,t.dataincrement_KB AS Data_Increment_KB
        ,IIF(t.support4G = 1, 'Yes', 'No') AS LTE,tc.value AS TADIG FROM Tariffs t
        INNER JOIN TadigCodes tc
        ON t.OperatorID = tc.OperatorID
        WHERE t.tarifflistid = {0}
        '''.format(self.roamdb_tarifflistid)
        return sql_srt
    
    def upload_data_from_ocs(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')
        sql_srt = self.ocs_tariff_gen()
        self.df_ocs_cost = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def upload_data_from_roamdb(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')
        sql_srt = self.roamdb_gen()
        self.df_roamdb = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def prepare_pp(self):
        '''Функция для подготовки DF из файла.
        Объединение с TADIG, COUNTRY и NETWORK_ID из OCS.
        Фильтрация по максимальному тарифу передачи данных.'''
        df_temp = pd.read_csv(join(self.downloads, self.tariff_file_name))
        df_temp = df_temp[df_temp['DATA'] <= discount_data_price_threshold]
        df_temp = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        self.df_tariff = df_temp

In [4]:
#######################################################################################
# Define variables here
#######################################################################################

dmi_allowed_list = 'DMI.dmi_allowed_list_export_Wed_Jan_22_2020_after.csv'
combined_dict = {'Partner_Combined':'S1', 'P4_Combined':'S2', 'MB_Combined':'S4'}
discount_data_price_threshold = 0.7 # USD
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'

reload_tariff_from_db = False # False if the tariffs should be taken from files

# Output file names
file_compare_by_country_coverage = 'CompareByCountryCoverage.csv'
file_compare_by_lte_coverage = 'CompareByLteCoverage.csv'
file_compare_tariffs = 'CompareTariffs.csv'
file_country_min_tariffs = 'CountryMinTariff.xlsx'
file_country_min_tariffs_lte = 'CountryMinTariffLte.xlsx'
file_operator_min_tariffs_lte = 'OperatorMinTariffLte.xlsx'

# Files with the tariffs should contain the following fields:
# TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE

# name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid, reload_tariff_from_db, downloads, tariff_file_name
s1_sponsor = SponsorClass('S1', 1, 267, 35, tariff_file_name = 'S1_20200101.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s2_sponsor = SponsorClass('S2', 2, 268, 34, tariff_file_name = 'S2_20200201_FULL_EUR.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s4_sponsor = SponsorClass('S4', 4, 329, 36, tariff_file_name = 'S4_20200101.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s5_sponsor = SponsorClass('S5', 5, 343, 32, tariff_file_name = 'S5_20200101_FULL_RB_COST.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s8_sponsor = SponsorClass('S8', 8, 372, 37, tariff_file_name = 'S8_20200101.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)

# sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s8_sponsor]
sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s8_sponsor]

#######################################################################################

#######################################################################################

In [5]:
for sponsor in sponsors:
    print(f'\nTariffs of {sponsor.name}:')
    display(sponsor.df_tariff.head(3))


Tariffs of S1:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,COUNTRY,NETWORK,NETWORK_ID
0,ALBVF,0.0300,0.0,0.0060,0.0024,1,1.0,Yes,Yes,Albania,Vodafone,12.0
1,ANDMA,0.0900,0.0,0.0300,0.0300,1,1.0,No,No,Andorra,Andorra Telecom S A U,17.0
2,AIACW,0.1606,0.0,0.0128,0.0257,60,1.0,No,Yes,Anguilla,Cable and Wireless,20.0



Tariffs of S2:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,CAMEL,DailyCap,DailyCapLimit,COUNTRY,NETWORK,NETWORK_ID
0,ALBAM,0.449,0,0.052,0.040,1,1,YES,YES,NaN,NaN,Albania,TELEKOM ALBANIA SH A,9.0
1,ALBEM,0.390,0,0.040,0.039,60,1,YES,YES,NaN,NaN,Albania,Eagle Mobile,10.0
2,ARGTM,NaN,0,0.143,0.040,60,1,NO,YES,NaN,NaN,Argentina,Movistar/Telefonica,27.0



Tariffs of S4:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,COUNTRY,NETWORK,NETWORK_ID
0,HKGH3,0.013,0.013,0.13,0.00315,60,1,Yes,Hong Kong (China),H3G/Hutchinson,392.0
1,HKGHT,0.013,0.013,0.13,0.00315,60,1,Yes,Hong Kong (China),H3G/Hutchinson,392.0
2,MACHT,0.130,0.130,0.13,0.00504,60,1,Yes,Macao China,Hutchison Telephone Macau Company Limited,567.0



Tariffs of S5:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,COUNTRY,NETWORK,NETWORK_ID
0,AFGAW,NaN,0.1725,0.115000,0.115000,60,1,No,Afghanistan,Afghan Wireless Communication Company,4.0
1,AFGEA,NaN,0.0000,0.017250,0.017250,60,1,No,Afghanistan,Etisalat,6.0
2,ALBVF,0.161,0.0000,0.008625,0.008625,60,1,No,Albania,Vodafone,12.0



Tariffs of S8:


,TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE,COUNTRY,NETWORK,NETWORK_ID
0,ABWDC,0.3576,0.0,0.1800,0.0720,NaN,1,No,Aruba,New millennium Telcom services,1160.0
1,AFGAR,1.0164,0.0,0.1236,0.1044,NaN,1,No,Afghanistan,Areeba/MTN,5.0
2,AFGEA,0.7956,0.0,0.2880,0.2976,NaN,1,No,Afghanistan,Etisalat,6.0


In [6]:
# Загрузить Combined Allowed list

df_al=pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_AL','Sponsor':'SPONSOR'}, inplace=True, axis=1)
df_al = df_al[df_al.SPONSOR.isin(list(combined_dict))]
df_al = df_al.replace(combined_dict)
df_al = pd.merge(df_al, df_tadig[['COUNTRY','TADIG','NETWORK_ID']], how='left', left_on='TADIG_AL', right_on='TADIG')
df_al.drop('TADIG', inplace=True,axis=1)

# Проверить дублирования стран в Combined Allowed List

s_temp = df_al.groupby('COUNTRY')['SPONSOR'].nunique()
if s_temp[s_temp > 1].count():
    print('Duplicates per Country in the Combined Allowed List:')
    display(s_temp[s_temp > 1])
else:
    print('No duplicates per Country found in the Combined Allowed List.')

df_al.head(3)

No duplicates per Country found in the Combined Allowed List.


,SPONSOR,TADIG_AL,COUNTRY,NETWORK_ID
0,S2,HRVT2,Croatia,225.0
1,S2,CZEET,Czech Republic,234.0
2,S2,HRVVI,Croatia,226.0


In [7]:
# Получаем данные по Usage из BSS

start_date = '20191201'
end_date = '20200101'

sql_srt="""
SELECT RepDate,SponsorPrefix,VisitedNetworkId AS NETWORK_ID,UsageType,SUM(Cost) Cost, SUM(Revenue) Revenue, SUM(Volume) Volume FROM
(
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,Volume
,'DATA' AS UsageType
FROM aggreg.GPRS_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,NbEvent AS Volume
,CASE WHEN Sms = 0 THEN 'MOC' WHEN Sms = 1 THEN 'MOSMS' END AS UsageType
FROM aggreg.MOC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
UNION ALL
SELECT
DATE_FORMAT(Day, "%Y-%m") AS RepDate,SponsorPrefix,VisitedNetworkId,Customer,Cost,Revenue,NbEvent AS Volume
,CASE WHEN Sms = 0 THEN 'MTC' WHEN Sms = 1 THEN 'MTSMS' END AS UsageType
FROM aggreg.MTC_AGR
WHERE `Day` BETWEEN {start_date} AND {end_date}
) t GROUP BY RepDate,SponsorPrefix,VisitedNetworkId,UsageType
""".format(start_date=start_date, end_date=end_date)

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_source = pd.read_sql_query(sql_srt, cnxn)

# sql_srt='SELECT network_id AS VisitedNetworkId, MAX(tadig_code) AS Tadig FROM RDB_TADIG_CODES GROUP BY network_id;'
# with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
#     df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_bss_source.SponsorPrefix.replace({42501:1, 26006:2, 45403:4, 51503:5}, inplace=True)
df_bss_source = df_bss_source.pivot_table(values='Volume',index=['RepDate','SponsorPrefix','NETWORK_ID']
                   ,columns='UsageType', aggfunc=np.sum).reset_index()

df_bss_processed = pd.merge(df_bss_source, df_tadig[['COUNTRY','NETWORK_ID']].drop_duplicates(), how='left', on='NETWORK_ID')
df_bss_processed = df_bss_processed[['RepDate','SponsorPrefix','COUNTRY','NETWORK_ID','MOC','MTC','MOSMS','MTSMS','DATA']]
df_bss_processed.DATA = df_bss_processed.DATA/1024/1024/1024
df_bss_processed.head(3)

,RepDate,SponsorPrefix,COUNTRY,NETWORK_ID,MOC,MTC,MOSMS,MTSMS,DATA
0,2019-12,1,Albania,12,342.0,124.0,8.0,32.0,13.385620
1,2019-12,1,Anguilla,20,NaN,NaN,NaN,NaN,0.060920
2,2019-12,1,Antigua and Barbuda,23,1.0,NaN,10.0,NaN,0.923297


### Cравнение покрытия по странам

In [8]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['DATA'].min().rename({'DATA':f'DATA_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')

df = df[df.isnull().any(axis=1)]
df.to_csv(join(downloads, file_compare_by_country_coverage), index=False)
df.head()

,COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S8
0,Albania,0.0024,0.039,NaN,0.008625,0.018
1,Andorra,0.0300,NaN,NaN,NaN,NaN
2,Anguilla,0.0257,NaN,NaN,NaN,0.126
3,Antigua and Barbuda,0.0240,NaN,NaN,NaN,0.126
4,Argentina,0.0140,0.040,NaN,0.009200,0.054


### Сравнение покрытия по LTE

In [9]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['LTE'].max().rename({'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')

df = df[(df == 'Yes').any(axis=1) & (df == 'No').any(axis=1)]
df.to_csv(join(downloads, file_compare_by_lte_coverage), index=False)
df.head()

,COUNTRY,LTE_S1,LTE_S2,LTE_S4,LTE_S5,LTE_S8
0,Albania,Yes,YES,NaN,No,Yes
2,Anguilla,No,NaN,NaN,NaN,Yes
3,Antigua and Barbuda,No,NaN,NaN,NaN,Yes
5,Armenia,No,NaN,NaN,No,Yes
8,Azerbaijan,Yes,NO,NaN,No,No


### Сравнение тарифов по странам

In [10]:
df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['DATA'].min().rename({'DATA':f'DATA_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp, how='outer', on='COUNTRY')
    
df = pd.merge(df, df_bss_processed.groupby('COUNTRY', as_index=False)['DATA'].sum(), how='left', on='COUNTRY')

df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_country_compare = df.sort_values(by='DATA', ascending=False)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #81EF21' if v else '' for v in is_max]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

df_country_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
.to_excel(join(downloads, file_country_min_tariffs), 'Sheet1', index=False)

display(df_country_compare.head(20).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1).hide_index())

COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S8,DATA,SPONSOR
Italy,0.0018,0.0026,nan,0.00863,0.0048,2395.96,S1
United Kingdom,0.0016,0.0012,nan,0.00575,0.0048,2059.13,S2
Germany,0.0024,0.00105,nan,0.00863,0.0072,1879.11,S2
Spain,0.0024,0.0018,nan,0.00863,0.012,1865.8,S2
South Korea,0.0016,0.0058,0.00315,0.0023,0.0324,1642.27,S1
France,0.0038,0.0014,nan,0.00132,0.0072,1463.33,S2
Russian Federation,0.004,0.03,0.01008,0.0115,0.018,1165.84,S1
United States,nan,0.003,0.00504,0.00575,0.018,1041.07,S2
Australia,0.0022,0.008,0.00504,0.0023,0.0072,926.205,S1
China,0.005,0.015,0.0063,0.00288,0.0144,885.221,S1


### Новые страны для Combined Allowed List

In [15]:
# Новые страны для Combined Allowed List

df_country_compare[df_country_compare.SPONSOR.isna()]

,COUNTRY,DATA_S1,DATA_S2,DATA_S4,DATA_S5,DATA_S8,DATA,SPONSOR
33,El Salvador,0.0140,NaN,NaN,0.009200,0.0540,7.931016,NaN
132,Malaysia,NaN,NaN,0.0100,0.006612,0.0144,7.835511,NaN
150,Pakistan,NaN,NaN,NaN,0.230000,0.0096,1.814714,NaN
54,Ivory Coast,0.0566,NaN,NaN,0.011500,0.1044,1.201465,NaN
85,Nigeria,0.0250,NaN,NaN,0.011500,0.0636,0.402604,NaN
121,Venezuela,0.2000,NaN,NaN,NaN,0.0528,0.018228,NaN
11,Benin,0.0700,NaN,NaN,NaN,NaN,NaN,NaN
35,Ethiopia,0.2500,NaN,NaN,NaN,NaN,NaN,NaN
46,Honduras,0.0250,NaN,NaN,0.009200,0.0540,NaN,NaN
63,Lesotho,0.0024,NaN,NaN,0.008625,0.1776,NaN,NaN


### Пропавшие и новые сети для Combined Allowed List

In [16]:
fields_list = ['COUNTRY','TADIG','TADIG_AL','DATA','SPONSOR']
df_new_mis_operatirs = DataFrame()

for sponsor in sponsors:
    df_sponsor = pd.merge(sponsor.df_tariff, df_al[df_al.SPONSOR == sponsor.name],
                       how='outer', on=['COUNTRY','NETWORK_ID'])[fields_list]
    s_sponsor = df_sponsor.loc[df_sponsor.SPONSOR == sponsor.name, 'COUNTRY'].unique()
    df_sponsor = df_sponsor.loc[df_sponsor.COUNTRY.isin(s_sponsor)]
    df_mis = df_sponsor.loc[df_sponsor.TADIG.isna()].copy()
    df_mis['COM'] = 'MIS'
    df_new = df_sponsor.loc[df_sponsor.SPONSOR.isna()].copy()
    df_new['COM'] = 'NEW'
    df_new['SPONSOR'] = sponsor.name
    df_new_mis_operatirs = df_new_mis_operatirs.append(df_mis).append(df_new)
    
df_new_mis_operatirs.sort_values(by='COUNTRY')

,COUNTRY,TADIG,TADIG_AL,DATA,SPONSOR,COM
0,Albania,ALBAM,NaN,0.009000,S1,NEW
4,Argentina,ARGCM,NaN,0.025000,S1,NEW
270,Argentina,ARGVO,NaN,0.025000,S1,NEW
269,Argentina,ARGTP,NaN,0.025000,S1,NEW
23,Brazil,BRACL,NaN,0.025000,S1,NEW
40,Chile,CHLSM,NaN,0.025000,S1,NEW
65,Ecuador,ECUPG,NaN,0.025000,S1,NEW
54,Germany,DEUE1,NaN,0.001207,S2,NEW
293,India,INDIB,NaN,0.020000,S1,NEW
119,Indonesia,IDN89,NaN,0.015000,S1,NEW


### Выгрузка полного сравнительного листа в файл

In [11]:
# Выгрузка полного сравнительного листа в файл

df = DataFrame({'COUNTRY':[],'NETWORK_ID':[],'NETWORK':[],'TADIG':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.rename({'MOC':f'MOC_{sponsor.name}'
                                        ,'MTC':f'MTC_{sponsor.name}'
                                        ,'SMS':f'SMS_{sponsor.name}'
                                        ,'DATA':f'DATA_{sponsor.name}'
                                        ,'LTE':f'LTE_{sponsor.name}'}
                                        , axis=1).drop(['Voice_Increment_SEC','Data_Increment_KB'], axis=1)
    df = pd.merge(df, df_temp, how='outer', on=['COUNTRY','NETWORK_ID','NETWORK','TADIG'])
    
cols = ['COUNTRY','NETWORK','TADIG']
for sponsor in sponsors:
    cols.extend([f'MOC_{sponsor.name}',f'MTC_{sponsor.name}',f'SMS_{sponsor.name}',f'DATA_{sponsor.name}',f'LTE_{sponsor.name}'])

df[cols].to_csv(join(downloads, file_compare_tariffs), index=False)
df[cols].head()

,COUNTRY,NETWORK,TADIG,MOC_S1,MTC_S1,SMS_S1,DATA_S1,LTE_S1,MOC_S2,MTC_S2,...,MOC_S5,MTC_S5,SMS_S5,DATA_S5,LTE_S5,MOC_S8,MTC_S8,SMS_S8,DATA_S8,LTE_S8
0,Albania,Vodafone,ALBVF,0.0300,0.0,0.0060,0.0024,Yes,NaN,NaN,...,0.161,0.000,0.008625,0.008625,No,0.2460,0.0000,0.1020,0.0204,Yes
1,Andorra,Andorra Telecom S A U,ANDMA,0.0900,0.0,0.0300,0.0300,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.3576,0.0000,0.2388,0.1260,Yes
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,No,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.3576,0.0000,0.2388,0.1260,Yes
4,Argentina,Movistar/Telefonica,ARGTM,0.0600,0.0,0.0150,0.0140,No,NaN,0.0,...,0.690,0.115,0.115000,0.115000,No,1.6020,0.2676,0.3576,0.0540,Yes


### Сравнение тарифов по странам с подкраской LTE

In [17]:
# Подготовка DF для сравнения тарифов по странам с учетом LTE

df = DataFrame({'COUNTRY':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.loc[:, ['COUNTRY', 'DATA', 'LTE']]
    df_temp = df_temp.loc[df_temp.groupby('COUNTRY')['DATA'].idxmin()]\
    .rename({'DATA':f'DATA_{sponsor.name}', 'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp.drop_duplicates(), how='outer', on='COUNTRY')

df = pd.merge(df, df_bss_processed.groupby('COUNTRY', as_index=False)['DATA'].sum(), how='left', on='COUNTRY')

df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_country_compare = df.sort_values(by='DATA', ascending=False)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #e6ffe6' if v else '' for v in is_max]

def highlight_lte(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_lte = s == 'Yes'
    lte_list = ['color: #cc3300' if v else '' for v in is_lte]
    lte_list.append('')
    return lte_list[1:]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

display(df_country_compare.head(15).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).hide_index())

df_country_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).to_excel(join(downloads, file_country_min_tariffs_lte), 'Sheet1', index=False)

COUNTRY,DATA_S1,LTE_S1,DATA_S2,LTE_S2,DATA_S4,LTE_S4,DATA_S5,LTE_S5,DATA_S8,LTE_S8,DATA,SPONSOR
Italy,0.0018,No,0.00299,No,nan,nan,0.00862,Yes,0.0048,No,2395.96,S1
United Kingdom,0.0016,Yes,0.00138,Yes,nan,nan,0.00575,No,0.0048,Yes,2059.13,S2
Germany,0.0024,Yes,0.00121,No,nan,nan,0.00862,Yes,0.0072,Yes,1879.11,S2
Spain,0.0024,Yes,0.00207,Yes,nan,nan,0.00862,Yes,0.012,Yes,1865.8,S2
South Korea,0.0016,Yes,0.00667,nan,0.00315,Yes,0.0023,Yes,0.0324,Yes,1642.27,S1
France,0.0038,Yes,0.00161,Yes,nan,nan,0.00132,Yes,0.0072,Yes,1463.33,S2
Russian Federation,0.004,Yes,0.0345,Yes,0.01,Yes,0.0115,Yes,0.018,Yes,1165.84,S1
United States,nan,nan,0.00345,Yes,0.005,Yes,0.00575,Yes,0.018,Yes,1041.07,S2
Australia,0.0022,Yes,0.0092,Yes,0.005,Yes,0.0023,No,0.0072,Yes,926.205,S1
China,0.005,Yes,0.01725,nan,0.0063,Yes,0.00288,Yes,0.0144,Yes,885.221,S1


### Сравнение тарифов по операторам с подкраской LTE

In [12]:
# Сравнение тарифов по операторам с подкраской LTE

df = DataFrame({'COUNTRY':[], 'NETWORK':[],'NETWORK_ID':[]})
for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby(['COUNTRY', 'NETWORK','NETWORK_ID'], as_index=False)[['DATA', 'LTE']].first()\
    .rename({'DATA':f'DATA_{sponsor.name}', 'LTE':f'LTE_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp.drop_duplicates(), how='outer', on=['COUNTRY','NETWORK','NETWORK_ID'])

df = pd.merge(df, df_bss_processed[['NETWORK_ID','DATA']], how='left', on='NETWORK_ID')
df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_operator_compare = df.sort_values(by='DATA', ascending=False)
df_operator_compare.drop(['NETWORK_ID'], axis=1, inplace=True)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #e6ffe6' if v else '' for v in is_max]

def highlight_lte(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_lte = s == 'Yes'
    lte_list = ['color: #cc3300' if v else '' for v in is_lte]
    lte_list.append('')
    return lte_list[1:]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

display(df_operator_compare.head(15).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).hide_index())

df_operator_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).to_excel(join(downloads, file_operator_min_tariffs_lte), 'Sheet1', index=False)

COUNTRY,NETWORK,DATA_S1,LTE_S1,DATA_S2,LTE_S2,DATA_S4,LTE_S4,DATA_S5,LTE_S5,DATA_S8,LTE_S8,DATA,SPONSOR
Italy,Wind Telecomunicazioni S p A,0.0018,Yes,0.0026,NO,nan,nan,nan,nan,0.0048,No,1734.83,S1
South Korea,KT Corporation,0.0016,Yes,0.04,YES,0.00315,Yes,0.00575,Yes,0.0324,Yes,1597.11,S1
Germany,O2,0.013,No,0.00105,YES,nan,nan,0.023,No,0.0168,Yes,1162.04,S2
France,Bouygues T l com,0.0038,Yes,0.0014,YES,nan,nan,0.023,Yes,0.018,Yes,1002.94,S2
United States,AT&T Mobility,nan,nan,0.003,YES,0.00504,Yes,0.00575,Yes,0.018,Yes,994.737,S2
Spain,Telefonica - TE,0.013,No,0.0018,YES,nan,nan,0.023,No,0.0168,Yes,919.165,S2
Australia,Singtel Optus,0.0022,Yes,0.008,YES,nan,nan,0.0023,No,0.0072,Yes,889.87,S1
China,China Mobile GSM,0.005,Yes,0.015,YES,nan,nan,0.00288,Yes,0.0144,Yes,846.48,S1
Spain,Vodafone,0.0024,Yes,0.0047,NO,nan,nan,0.00863,Yes,0.0204,Yes,824.571,S2
Turkey,Vodafone Telekomunikasyon A S,0.0024,Yes,nan,nan,nan,nan,0.00863,Yes,0.0204,Yes,713.367,S1


### Подготовить файлы со всеми тарифами из файла для загрузки в OCS

In [16]:
# Подготовить файлы со всеми тарифами для загрузки в OCS

tariff_start_date = '2019-12-01 00:00:00'
charge_cost_increment = 2

def create_file_upload_to_ocs(sponsor_obj):
    df = sponsor_obj.df_tariff.copy()
    df['ROAMING_PLAN_RULE_ID'] = ''
    df['ROAMING_PLAN_ID']=''
    df['SPONSOR_ID'] = sponsor_obj.sponsor_id
    df['MT_SMS_RATE'] = 0
    df['CURRENCY_ID'] = 1 #1:USD, 2:EUR
    df['START_DATE'] = tariff_start_date
    df['END_DATE'] = 'NULL'
    df['is_discounted'] = 1
    return df

for sponsor in sponsors:
    df_temp = create_file_upload_to_ocs(sponsor)
    df_temp[['ROAMING_PLAN_RULE_ID',
            'ROAMING_PLAN_ID',
            'NETWORK_ID',
            'SPONSOR_ID',
            'MOC',
            'MTC',
            'SMS',
            'MT_SMS_RATE',
            'DATA',
            'CURRENCY_ID',
            'START_DATE',
            'END_DATE',
            'is_discounted']].drop_duplicates().to_csv(join(downloads, f'ocs_insert_{sponsor.name}_cost.csv'), index=False)
    
    df_temp[['MOC','MTC','SMS','DATA']] = df_temp[['MOC','MTC','SMS','DATA']] * charge_cost_increment
    
    df_temp[['ROAMING_PLAN_RULE_ID',
            'ROAMING_PLAN_ID',
            'NETWORK_ID',
            'SPONSOR_ID',
            'MOC',
            'MTC',
            'SMS',
            'MT_SMS_RATE',
            'DATA',
            'CURRENCY_ID',
            'START_DATE',
            'END_DATE',
            'is_discounted']].drop_duplicates().to_csv(join(downloads, f'ocs_insert_{sponsor.name}_charge.csv'), index=False)

### Подготовить файлы со всеми тарифами из файла для загрузки в RoamDB

In [9]:
# Подготовить файлы со всеми тарифами для загрузки в RoamDB

columns_origin = ['TADIG',
  'MOC',
  'MTC',
  'SMS',
  'DATA',
  'Voice_Increment_SEC',
  'Data_Increment_KB',
  'IsDiscounted (Yes/No)',
  'LTE']

columns_roamdb = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

dict_roamdb_columns = dict(zip(columns_origin,columns_roamdb))

def create_file_upload_to_roamdb(sponsor_obj):
    df = sponsor_obj.df_tariff.copy()
    df['IsDiscounted (Yes/No)'] = 'Yes'
    return df.rename(dict_roamdb_columns, axis=1)

for sponsor in sponsors:
    df_temp = create_file_upload_to_roamdb(sponsor)
    df_temp[columns_roamdb].drop_duplicates().to_csv(join(downloads, f'roamdb_upload_{sponsor.name}.csv'), index=False)

Для загрузки в RoamDb удалить все тарифы спонсора из тарифа

USE [RoamDb];
GO

SELECT * FROM sys.tables ORDER BY name

SELECT * FROM TariffLists
SELECT * FROM Tariffs

BEGIN TRAN

DELETE FROM Tariffs WHERE TariffListID = 

--COMMIT TRAN
--ROLLBACK TRAN

SELECT * FROM Tariffs WHERE TariffListID = 

### Разработки

In [35]:
# Сравнение тарифов по операторам с подкраской LTE и с добавлением CAMEL

df = DataFrame({'COUNTRY':[], 'NETWORK':[],'NETWORK_ID':[]})

sponsors = [s5_sponsor, s8_sponsor]

for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby(['COUNTRY', 'NETWORK','NETWORK_ID'], as_index=False)[['DATA', 'LTE', 'CAMEL']].first()\
    .rename({'DATA':f'DATA_{sponsor.name}', 'LTE':f'LTE_{sponsor.name}', 'CAMEL':f'CAMEL_{sponsor.name}'}, axis=1)
    df = pd.merge(df, df_temp.drop_duplicates(), how='outer', on=['COUNTRY','NETWORK','NETWORK_ID'])

df = pd.merge(df, df_bss_processed[['NETWORK_ID','DATA']], how='left', on='NETWORK_ID')
df = pd.merge(df, df_al.groupby('COUNTRY', as_index=False)['SPONSOR'].first(), how='outer', on='COUNTRY')

df_operator_compare = df.sort_values(by='DATA', ascending=False)
df_operator_compare.drop(['NETWORK_ID'], axis=1, inplace=True)

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: #e6ffe6' if v else '' for v in is_max]

def highlight_lte(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_lte = s == 'Yes'
    lte_list = ['color: #cc3300' if v else '' for v in is_lte]
    lte_list.append('')
    return lte_list[1:]

data_tariff_name_list = [f'DATA_{sponsor.name}' for sponsor in sponsors]

display(df_operator_compare.head(15).round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).hide_index())

df_operator_compare.round(5).style.apply(highlight_max, subset=data_tariff_name_list, axis=1)\
        .apply(highlight_lte, axis=1).to_excel(join(downloads, file_operator_min_tariffs_lte), 'Sheet1', index=False)

COUNTRY,NETWORK,DATA_S5,LTE_S5,CAMEL_S5,DATA_S8,LTE_S8,CAMEL_S8,DATA,SPONSOR
Italy,Wind Telecomunicazioni S p A,nan,nan,nan,0.0048,No,Yes,1719.77,S1
South Korea,KT Corporation,nan,nan,nan,0.0324,Yes,Yes,1149.36,S1
China,China Mobile GSM,0.00805,Yes,Yes,0.0144,Yes,No,939.868,S1
Germany,O2,0.023,No,No,0.0168,Yes,Yes,796.381,S2
United States,AT&T Mobility,0.00575,Yes,Yes,0.018,Yes,Yes,755.378,S2
Spain,Vodafone,0.00862,Yes,No,0.0204,Yes,Yes,736.383,S2
Spain,Telefonica - TE,0.023,No,Yes,0.0168,Yes,Yes,732.528,S2
Germany,Vodafone GmbH,0.00862,Yes,No,0.0204,Yes,Yes,706.918,S2
Turkey,Vodafone Telekomunikasyon A S,0.00862,Yes,No,0.0204,Yes,Yes,702.001,S2
South Africa,Vodacom Group Pty Ltd,0.00862,Yes,No,0.0204,Yes,Yes,663.275,S1
